In [3]:
from pyngrok import ngrok
import streamlit as st

# Write your Streamlit script in an app.py file
with open("app.py", "w") as f:
    f.write("""
import google.generativeai as genai
import requests
import gradio as gr
import streamlit as st


def get_Weather(city):
      '''
      Fetches weather data from OpenWeatherMap API.

      Parameters:
      city (str): Name of the city to get weather data for.

      Returns:
      dict: JSON response containing weather details such as temperature,
            humidity, wind speed, sunrise/sunset times, etc.
      '''

      url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&units=metric&appid=YOUR_WEATHER_API_KEY"
      r = requests.get(url)
      if r.status_code == 200:
            return r.json()
      return None

import time
import google.generativeai as genai
from datetime import datetime, timezone, timedelta


def unix_to_local_time(unix_timestamp, timezone_offset_seconds):
    '''
    Converts a Unix timestamp to a human-readable local time string.

    Parameters:
    - unix_timestamp (int): The Unix timestamp (seconds since Jan 1, 1970 UTC).
    - timezone_offset_seconds (int): The timezone offset in seconds (e.g., +3600 for UTC+1).

    Returns:
    - str: Local time formatted as "HH:MM AM/PM".

    Example:
        unix_timestamp = 1754832000  # corresponds to 2025-08-10 12:00:00 UTC
        timezone_offset_seconds = 7200  # +2 hours offset

        local_time = unix_to_local_time(unix_timestamp, timezone_offset_seconds)
        print(local_time)  # Output: "02:00 PM"
    '''

    local_time = datetime.utcfromtimestamp(unix_timestamp) + timedelta(seconds=timezone_offset_seconds)
    return local_time.strftime("%I:%M %p")



_cache = {}
CACHE_DURATION = 15 * 60

def generate_weather_report(city):
    '''
    Generates a weather report for a given city, using cached data if available and recent.

    Parameters:
    - city (str): Name of the city to get the weather for.

    Returns:
    - str: A narrative weather report with practical advice, generated by an AI model.

    How it works:
    1. Normalizes the city name to lowercase and trims spaces.
    2. Checks if the report for this city is cached and not older than CACHE_DURATION seconds.
    3. If cached and fresh, returns the cached report.
    4. Otherwise, calls the weather API to get current data.
    5. Converts sunrise and sunset times from Unix timestamps to local time strings.
    6. Builds a prompt with weather data for the AI to generate a narrative report.
    7. Caches the generated report for future use.
    8. Returns the AI-generated weather report.
    '''

    city_key = city.strip().lower()
    current_time = time.time()

    if city_key in _cache:
        cached_time, cached_response = _cache[city_key]
        if current_time - cached_time < CACHE_DURATION:
            return cached_response

    genai.configure(api_key="YOUR_GOOGLE_API_KEY")

    data = get_Weather(city)

    if not data or "name" not in data:
        return " City not found. Please check the name and try again."

    template = '''
Here is the weather data for the city of {city}:
- Current temperature: {temp}°C
- Feels like: {feels_like}°C
- Minimum temperature: {temp_min}°C
- Maximum temperature: {temp_max}°C
- Pressure: {pressure} hPa
- Humidity: {humidity}%
- Weather description: {description}
- Wind speed: {wind_speed} m/s
- Cloud coverage: {clouds}%
- Sunrise time: {sunrise}
- Sunset time: {sunset}

Could you please write a clear and pleasant narrative weather report in the style of a weather news broadcast?

Also, please provide practical advice on suitable clothing and activities.

Report:
'''

    timezone_offset = data.get("timezone", 0)

    params = {
        "city": data["name"],
        "description": data["weather"][0]["description"],
        "temp": data["main"]["temp"],
        "feels_like": data["main"]["feels_like"],
        "temp_min": data["main"]["temp_min"],
        "temp_max": data["main"]["temp_max"],
        "pressure": data["main"]["pressure"],
        "humidity": data["main"]["humidity"],
        "wind_speed": data["wind"]["speed"],
        "clouds": data["clouds"]["all"],
        "sunrise": unix_to_local_time(data["sys"]["sunrise"], timezone_offset),
        "sunset": unix_to_local_time(data["sys"]["sunset"], timezone_offset)
    }

    prompt = template.format(**params)

    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(prompt)

    _cache[city_key] = (current_time, response.text)

    return response.text





st.title("Bulletin météo IA")

city = st.text_input("Enter the city name")

if city:
    result = generate_weather_report(city)
    st.write(result)
""")

# Configure the ngrok token
ngrok.set_auth_token("YOUR_NGROK_TOKEN")

# Open a public tunnel on port 8501
public_url = ngrok.connect(8501)
print("URL publique pour accéder à Streamlit:", public_url)

# Run Streamlit in the background in Colab
get_ipython().system_raw("streamlit run app.py &")

URL publique pour accéder à Streamlit: NgrokTunnel: "https://cabce9daac6a.ngrok-free.app" -> "http://localhost:8501"


## Why Use Ngrok with Streamlit?

Streamlit apps run locally on your machine, typically accessible only via `localhost` (e.g., `http://localhost:8501`). This means **only you** can access the app from that computer.

**Ngrok** is a tool that creates a secure, public URL (a tunnel) that forwards to your local machine. This allows anyone with the link to access your Streamlit app over the internet, without deploying it to a public server.

In other words, ngrok acts as a temporary host for your local Streamlit app, making it accessible globally.

### Important Notes

- Ngrok’s free tier limits you to **3 active tunnels** at the same time.
- If you try to open more than 3 tunnels, ngrok will block new connections until some tunnels are closed.
- This is a limitation of the free plan, and paid plans offer higher limits and more features.

Using ngrok is a simple and quick way to share your Streamlit app with others during development or demos without needing to deploy it to a cloud service.
